In [38]:
%%capture
!pip install pyspark
!pip install findspark
!pip install pyngrok

In [39]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [40]:
spark = SparkSession.builder \
        .appName('testColab') \
        .getOrCreate()

In [42]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········
 * ngrok tunnel "https://e028-35-238-234-242.ngrok-free.app" -> "http://127.0.0.1:4040"


In [43]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [44]:
df_green = spark.read.parquet('gdrive/MyDrive/module5/pq/green/*/*')

In [45]:
df_green.registerTempTable('green')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [46]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [47]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [48]:
df_yellow = spark.read.parquet('gdrive/MyDrive/module5/pq/yellow/*/*')

In [49]:
df_yellow.registerTempTable('yellow')

In [32]:
df_green_revenue.write.parquet('gdrive/MyDrive/module5/report/revenue/green', mode='overwrite')

In [50]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [51]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('gdrive/MyDrive/module5/report/revenue/yellow', mode='overwrite')

In [52]:
df_green_revenue = spark.read.parquet('gdrive/MyDrive/module5/report/revenue/green')
df_yellow_revenue = spark.read.parquet('gdrive/MyDrive/module5/report/revenue/yellow')

In [53]:
df_green_revenue_tmp= df_green_revenue\
    .withColumnRenamed('amount', 'green_amount')\
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp= df_yellow_revenue\
    .withColumnRenamed('amount', 'yellow_amount')\
    .withColumnRenamed('number_records', 'yellow_number_records')

In [54]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [56]:
df_join.write.parquet('gdrive/MyDrive/module5/report/revenue/total', mode='overwrite')

AnalysisException: [COLUMN_ALREADY_EXISTS] The column `yellow_amount` already exists. Consider to choose another name or rename the existing column.

In [63]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  14|              NULL|                NULL|               8.8|                    1|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  22|              15.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000002|                   45|
|2020-01-01 00:00:00|  25| 5

In [58]:
df_join = spark.read.parquet('gdrive/MyDrive/module5/report/revenue/total')

In [61]:
df_zones = spark.read.parquet('gdrive/MyDrive/module5/zones')

In [62]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [65]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [67]:
df_result.drop('LocationID', 'zone').write.parquet('gdrive/MyDrive/module5/tmp/revenue-zones')